Logs
- [2024/02/11]    
  A copy of chapter 8 from (Grus, 2019)

In [119]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go   # pip install plotly nbformat (and restart kernel)
import pandas as pd

from scratch.linear_algebra import LinearAlgebra as la
from scratch.linear_algebra import Vector
from typing import Callable, TypeVar, List, Iterator

In [120]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The Idea Behing Gradient Descent

We have to define a function that has a vector input and turns into a scalar.

In [121]:
x_data = np.linspace(-6, 6, 20)
y_data = np.linspace(-6, 6, 20)
z_data = x_data[:, np.newaxis]**2 + y_data[np.newaxis, :]**2

fig = go.Figure(data=[go.Surface(z=z_data, x=x_data, y=y_data,
                                 colorscale="Viridis")])
r = 1.5
camera = {"eye": {"x": r, "y": r, "z": r}}
fig.update_layout(autosize=False, width=500, height=400, 
                  margin={"l":10, "r":10, "b":10, "t":10},
                  scene_camera=camera)

fig.show()

## Estimating the Gradient

$$
  f'(x) = \lim_{h \rightarrow 0} \frac{f(x + h) - f(x)}{h}
$$

In [122]:
def difference_quotient(f: Callable[[float], float], x: float, h:float) -> float:
  return (f(x + h) - f(x)) / h

$$
\begin{align*}
  \frac{\partial f(\vec{v})}{\partial v_i}
    &=\frac{\partial f(v_1, v_2, \ldots, v_j, \ldots, v_n)}{\partial v_i} \\
    &= \lim_{h \rightarrow  0} 
      \frac{f(v_1, v_2, \ldots, v_i + h, \ldots, v_n)
            - f(v_1, v_2, \ldots, v_i, \ldots, v_n)}{h}
\end{align*}
$$
$n$ is the dimension of vector $\vec{v}$

In [123]:
def partial_difference_quotient(f: Callable[[Vector], float],
                                v: Vector,
                                i: int,
                                h: float) -> float:
  """Returns the i-th partial difference quotient of f at v"""
  w = [v_j + (h if j == i else 0)     # add h to just the ith element of v
       for j, v_j in enumerate(v)]

  return (f(w) - f(v)) / h

$$
  \nabla f(\vec{v}) = \left(
    \frac{\partial f(\vec{v})}{\partial v_1},
    \frac{\partial f(\vec{v})}{\partial v_2},
    \ldots,
    \frac{\partial f(\vec{v})}{\partial v_i},
    \ldots,
    \frac{\partial f(\vec{v})}{\partial v_n}
  \right)^\mathsf{T}
$$

In [124]:
def estimate_gradient(f: Callable[[Vector], float],
                      v: Vector,
                      h: float = 0.0001) -> Vector:
  return [partial_difference_quotient(f, v, i, h) for i in range(len(v))]

Estimating gradient using difference quotient is very slow.  
It is better to provide an explicit formula for the derivative instead calculating  
from the difference quotient

## Using the Gradient

$$
  \vec{v}_{n+1} = \vec{v}_{n} + \gamma\nabla f(\vec{v}_n)
$$
$\gamma$ is a step size

Gradient of sum of squares
$$
\begin{align*}
  \nabla f(\vec{v}) = \nabla \left(\vec{v} \cdot \vec{v}\right)
    = \nabla\left(\sum_{j=1}^n v_j^2\right) 
    = \left(2 v_1, 2 v_2, \ldots, 2 v_n\right)^\mathsf{T}
\end{align*}
$$

In [125]:
def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
  """Moves `step_size` in the `gradient` direction from `v"""
  assert len(v) == len(gradient)
  step = la.scalar_multiply(step_size, gradient)

  return la.add(v, step)

def sum_of_squares_gradient(v: Vector) -> Vector:
  return [2 * v_i for v_i in v]

In [126]:
seed = 24_02_20
rng = np.random.default_rng(seed)
v = np.random.uniform(-10, 10, size=3) 
v

array([ 8.554546  , -2.42235269,  3.03460703])

In [127]:
# pick a random starting point
seed = 24_02_20
rng = np.random.default_rng(seed)
v = np.random.uniform(-10, 10, size=3) 

v_history = [v]
for epoch in range(1_000):
  grad = sum_of_squares_gradient(v)       # compute the gradient at v
  v = gradient_step(v, grad, -0.01)       # we step a negative step size = descent
  v_history.append(v)
  # print(epoch, v)

v_history = np.array(v_history)

assert la.distance(v, [0, 0, 0]) < 0.001


To understand this gradient descent procedure, let us
consider the problem in 2D dimensional vectors

In [128]:
# pick a random starting point
# seed = 24_02_19
# rng = np.random.default_rng(seed)
# v_2D = np.random.uniform(-10, 10, size=2)   # size determines the dimension of the vector
v_2D = np.array([-7, -7])

v_2D_history = [v_2D]
for epoch in range(1_000):
  grad = sum_of_squares_gradient(v_2D)       # compute the gradient at v_2D
  v_2D = gradient_step(v_2D, grad, -0.01)       # we step a negative step size = descent
  v_2D_history.append(v_2D)
  # print(epoch, v_2D)

v_2D_history = np.array(v_2D_history)

assert la.distance(v_2D, [0, 0]) < 0.001


In [129]:
v_2D_history

array([[-7.00000000e+00, -7.00000000e+00],
       [-6.86000000e+00, -6.86000000e+00],
       [-6.72280000e+00, -6.72280000e+00],
       ...,
       [-1.22665259e-08, -1.22665259e-08],
       [-1.20211954e-08, -1.20211954e-08],
       [-1.17807715e-08, -1.17807715e-08]])

[You do not need to understand the following code]

In [130]:
vx_data = np.linspace(-10, 10, 40)
vy_data = np.linspace(-10, 10, 40)
vx_mesh, vy_mesh = np.meshgrid(vx_data, vy_data, indexing="xy")
z_data = vx_mesh**2 + vy_mesh**2


fig = go.Figure(data=[go.Surface(z=z_data, x=vx_data, y=vy_data,
                                 colorscale="Viridis")])

offset = 5
fig.add_trace(go.Scatter3d(x=v_2D_history[:, 0], y=v_2D_history[:, 1], 
                           z=v_2D_history[:, 0]**2 + v_2D_history[:, 1]**2 + offset,
                           mode="lines",
                           line={"color": "yellow", "width": 3}))
r = 1.5
camera = {"eye": {"x": r, "y": r, "z": r}}
fig.update_layout(autosize=False, width=500, height=400, 
                  margin={"l":10, "r":10, "b":10, "t":10},
                  scene_camera=camera)

fig.show()


## Choosing the Right Step Size

It is not a science but rather an art to choose the right step size.
The following are the popular options:
- Using a fixe step size
- Gradually shrinking the step size over time
- At each step, choosing the step size that minimizes the value of the   
  objective function

## Using Gradient to Fit Models

_loss function_ is a function to measure how well the model fits our data.

Example:
$$
  y = 20 x + 5, \qquad x \in [-50, 49]
$$

In [131]:
# x ranges from -50 to 49, y is always 20 * x  + 5
inputs = [(x, 20 * x + 5) for x in range(-50, 50)]
inputs

[(-50, -995),
 (-49, -975),
 (-48, -955),
 (-47, -935),
 (-46, -915),
 (-45, -895),
 (-44, -875),
 (-43, -855),
 (-42, -835),
 (-41, -815),
 (-40, -795),
 (-39, -775),
 (-38, -755),
 (-37, -735),
 (-36, -715),
 (-35, -695),
 (-34, -675),
 (-33, -655),
 (-32, -635),
 (-31, -615),
 (-30, -595),
 (-29, -575),
 (-28, -555),
 (-27, -535),
 (-26, -515),
 (-25, -495),
 (-24, -475),
 (-23, -455),
 (-22, -435),
 (-21, -415),
 (-20, -395),
 (-19, -375),
 (-18, -355),
 (-17, -335),
 (-16, -315),
 (-15, -295),
 (-14, -275),
 (-13, -255),
 (-12, -235),
 (-11, -215),
 (-10, -195),
 (-9, -175),
 (-8, -155),
 (-7, -135),
 (-6, -115),
 (-5, -95),
 (-4, -75),
 (-3, -55),
 (-2, -35),
 (-1, -15),
 (0, 5),
 (1, 25),
 (2, 45),
 (3, 65),
 (4, 85),
 (5, 105),
 (6, 125),
 (7, 145),
 (8, 165),
 (9, 185),
 (10, 205),
 (11, 225),
 (12, 245),
 (13, 265),
 (14, 285),
 (15, 305),
 (16, 325),
 (17, 345),
 (18, 365),
 (19, 385),
 (20, 405),
 (21, 425),
 (22, 445),
 (23, 465),
 (24, 485),
 (25, 505),
 (26, 525),
 (27, 

$$
\begin{align*}
  \textrm{error}_i = \textrm{predicted}_i - y_i  = \textrm{slope} \cdot x_i + \textrm{intercept} - y_i, \qquad i = 1, \ldots, n
\end{align*}
$$

$$
\begin{align*}
  \textrm{mean squared error} 
    &= \frac{1}{n}\sum_{i=1}^n \left(\textrm{predicted}_i - y_i\right)^2 \\
  \nabla(f(\vec{v})) = \nabla\left( \textrm{mean squared error} \right)
    &= \left(\frac{\partial(\textrm{mean squared error})}{\partial (\textrm{slope})}, 
             \frac{\partial(\textrm{mean squared error})}{\partial (\textrm{intercept})}\right)^\mathsf{T} \\
    &= \left(\frac{1}{n}\sum_{i=1}^n 2\, (\textrm{predicted}_i - y_i) \, x_i, 
            \,\frac{1}{n}\sum_{i=1}^n (\textrm{predicted}_i - y_i) \right)^\mathsf{T} \\
    &= \frac{1}{n} \sum_{i=1}^n \left(2\,\textrm{error}_i \, x_i, 2\, \textrm{error}_i \right)^\mathsf{T}
    = \frac{1}{n}\sum_{i=1}^n 
      \begin{pmatrix}
        2\,\textrm{error}_i \, x_i\\
        2\, \textrm{error}_i 
      \end{pmatrix}
\end{align*}
$$

In this case, $\vec{v} = \vec{\theta} = (\textrm{slope}, \textrm{intercept})$

In [132]:
def linear_gradient(x: float, y: float, theta: Vector) -> Vector:
  """This is a gradient formula for squared error"""
  slope, intercept = theta
  predicted = slope * x + intercept   # the prediction of the model
  error = predicted - y               # error is (predicted - actual)
  squared_error = error**2            # we'll minimize squared error
  grad = [2 * error * x, 2 * error]   # using its gradient
  return grad

$$
  \vec{\theta}_{n+1} = \vec{\theta}_n + \gamma \nabla(f(\vec{\theta}))
$$

In [133]:
# Start with random values for slope and intercept
seed = 24_02_11
rng = np.random.default_rng(seed)
theta = [rng.uniform(-1, 1), rng.uniform(-1, 1)]    # starting value for theta

learning_rate = 0.001    # gamma

theta_history = [theta]
for epoch in range(5_000):
  # Compute the mean of the gradients
  grad = la.vector_mean([linear_gradient(x, y, theta) for x, y in inputs])

  # Take a step in that direction
  theta = gradient_step(theta, grad, -learning_rate)
  theta_history.append(theta)
  print(epoch, theta)

theta_history = np.array(theta_history)

slope, intercept = theta
assert 19.9 < slope < 20.1, "slope should be about 20"
assert 4.9 < intercept < 5.1, "intercept should be about 5"

0 [33.234627342714056, 0.6928780680840432]
1 [11.168196440477796, 0.7147269392905891]
2 [25.886527701140597, 0.7144656818524856]
3 [16.069400489021074, 0.7289232781899212]
4 [22.617438797101137, 0.7335348321225625]
5 [18.249901857165664, 0.7446852012554185]
6 [21.163060146471757, 0.7514457327100733]
7 [19.219990328036047, 0.7611059013911249]
8 [20.516027557101346, 0.7688036799163787]
9 [19.65157842309332, 0.7777821001136472]
10 [20.22817497389687, 0.7858781143365132]
11 [19.843593170525125, 0.794534533081737]
12 [20.100117889792823, 0.8027890571860986]
13 [19.929024156565372, 0.8112835969615192]
14 [20.043152171167858, 0.8195900539241615]
15 [19.96703709188496, 0.8279940259874811]
16 [20.017814253738717, 0.836305075027391]
17 [19.983954197831302, 0.8446502791310749]
18 [20.006547200325652, 0.8529449327706441]
19 [19.99148596231556, 0.8612455901054284]
20 [20.001540108725624, 0.8695145848875331]
21 [19.994842262064896, 0.8777770958264837]
22 [19.99931798829854, 0.8860163838968956]
23 [1

[You do not need to understand the following code]

In [134]:
vx_data = np.linspace(0, 35, 50)   # slope space
vy_data = np.linspace(0, 8, 50)   # intercept space
vx_mesh, vy_mesh = np.meshgrid(vx_data, vy_data, indexing="xy")
z_data = np.sum([(vx_mesh*x + vy_mesh - y)**2 for x, y in inputs], axis=0)/(len(inputs))
# print(np.shape(z_data))


fig = go.Figure(data=[go.Surface(z=z_data, x=vx_data, y=vy_data,
                                 colorscale="Viridis")])

offset = 1000 
start = 0
# print(np.shape(theta_history[start:, 0]))
z_gradient_descent_history = np.mean(
  [(theta_history[start:, 0]*x + theta_history[start:, 1] - y)**2 for x, y in inputs], 
  axis=0)
# print(np.shape(z_gradient_descent_history))
fig.add_trace(go.Scatter3d(x=theta_history[start:, 0], y=theta_history[start:, 1], 
                           z=z_gradient_descent_history + offset,
                           mode="lines",
                           line={"color": "red", "width": 3}))
r = 1.6
camera = {"eye": {"x": r, "y": r, "z": r}}
fig.update_layout(autosize=False, width=500, height=400,
                  scene={"xaxis_title": "slope",
                         "yaxis_title": "intercept",
                         "zaxis_title": "f(slope, intercept)"},
                  margin={"l":5, "r":10, "b":5, "t":10},
                  scene_camera=camera)

fig.show()



## Minibatch and Stochastic Gradient Descent

Minibatch gradiend descent. 

It can save much more space. Training for huge datasets.  
You only need to perform gradient update in each batch which has smaller size

In [135]:
T = TypeVar('T')      # this allows us to type "generic" functions

def minibatches(dataset: List[T],
                batch_size: int,
                shuffle: bool = True) -> Iterator[List[T]]:
  """Generates `batch_size`-sized minibatches from the dataset"""
  # start indexes 0, batch_size, 2 * batch_size, 3 * batch_size, ...
  batch_starts = [start for start in range(0, len(dataset), batch_size)]

  if shuffle: 
    rng = np.random.default_rng()
    rng.shuffle(batch_starts)       # shuffle the batches

  for start in batch_starts:
    end = start + batch_size
    yield dataset[start:end]

In [138]:
seed = 24_02_11
rng = np.random.default_rng(seed)
theta = [rng.uniform(-1, 1), rng.uniform(-1, 1)]

theta_minibatch_history = [theta]
for epoch in range(1_000):
  for batch in minibatches(inputs, batch_size=20):
    grad = la.vector_mean([linear_gradient(x, y, theta) for x, y in batch])
    theta = gradient_step(theta, grad, -learning_rate)
    theta_minibatch_history.append(theta)
  print(epoch, theta)

theta_minibatch_history = np.array(theta_minibatch_history)

slope, intercept = theta
assert 19.9 < slope < 20.1, "slope should be about 20"
assert 4.9 < intercept < 5.1, "intercept should be about 5"

0 [21.945826503223742, 0.8786119951218293]
1 [19.753407956007067, 0.949408570133191]
2 [20.066989474301455, 0.994513352614691]
3 [19.936673953592283, 1.036030014283585]
4 [19.994524963252207, 1.0753640933097923]
5 [20.004207144534913, 1.1146324371730532]
6 [20.032894508896167, 1.153170055606426]
7 [19.9458174833835, 1.1917725550288012]
8 [19.995258062972155, 1.2292473940456434]
9 [19.958172557698187, 1.2666739224680479]
10 [20.05776778110701, 1.3048199395786002]
11 [19.969343962704436, 1.3411772464906822]
12 [20.062539432559635, 1.378041441195186]
13 [19.94861371102906, 1.4144409640190048]
14 [20.056602568449254, 1.4512349935716242]
15 [19.94862313944184, 1.4868719753267419]
16 [19.99467213215404, 1.5217149064552504]
17 [20.02774366987701, 1.5570538214086158]
18 [19.97882182431353, 1.5921522959234904]
19 [20.02308763480502, 1.6261834315539299]
20 [20.032946948313143, 1.6598054757121568]
21 [20.02533885918361, 1.6931636853328405]
22 [19.960365925089338, 1.7261849939321747]
23 [19.966372

Stochastic gradient descent

Pick randomly each training data, and update the gradient.   
Sometimes, it produces much faster convergence solution.

In [152]:
seed = 24_02_12
rng = np.random.default_rng(seed)
theta = [rng.uniform(-1, 1), rng.uniform(-1, 1)]

# x ranges from -50 to 49, y is always 20 * x  + 5
inputs = [(x, 20 * x + 5) for x in range(-50, 50)]
rng.shuffle(inputs)

theta_sgd_history = [theta]
for epoch in range(100):
  for x, y in inputs:
    grad = linear_gradient(x, y, theta)
    theta = gradient_step(theta, grad, -learning_rate)
    theta_sgd_history.append(theta)
  print(epoch, theta)
theta_sgd_history = np.array(theta_sgd_history)

slope, intercept = theta
assert 19.9 < slope < 20.1, "slope should be about 20"
assert 4.9 < intercept < 5.1, "intercept should be about 5"

0 [20.56236295248587, 3.902726742495169]
1 [20.265069009315752, 4.482800347834722]
2 [20.124940235474604, 4.756218026031297]
3 [20.05889056016289, 4.885093405257957]
4 [20.02775805618699, 4.94583879480397]
5 [20.013083755378542, 4.974471124526222]
6 [20.006167026021277, 4.987966968596862]
7 [20.00290682673643, 4.994328232557796]
8 [20.001370132320943, 4.997326613316222]
9 [20.000645811652028, 4.998739899610856]
10 [20.000304403219683, 4.999406051881567]
11 [20.00014348040935, 4.999720042648643]
12 [20.000067629468212, 4.9998680421468675]
13 [20.000031877139012, 4.9999378016868645]
14 [20.00001502528438, 4.999970682834969]
15 [20.000007082165364, 4.999986181358911]
16 [20.00000333817749, 4.99999348658571]
17 [20.000001573449456, 4.999996929903205]
18 [20.000000741645177, 4.999998552910359]
19 [20.00000034957434, 4.999999317914513]
20 [20.000000164771823, 4.999999678499123]
21 [20.000000077665153, 4.999999848460613]
22 [20.000000036607492, 4.999999928571954]
23 [20.000000017254965, 4.999

In [157]:
vx_data = np.linspace(0, 35, 50)   # slope space
vy_data = np.linspace(0, 8, 50)   # intercept space
vx_mesh, vy_mesh = np.meshgrid(vx_data, vy_data, indexing="xy")
z_data = np.sum([(vx_mesh*x + vy_mesh - y)**2 for x, y in inputs], axis=0)/(len(inputs))
# print(np.shape(z_data))


fig = go.Figure(data=[go.Surface(z=z_data, x=vx_data, y=vy_data,
                                 colorscale="Viridis")])

offset = 1000 
start = 0
# print(np.shape(theta_history[start:, 0]))
z_gradient_descent_history = np.mean(
  [(theta_history[start:, 0]*x + theta_history[start:, 1] - y)**2 for x, y in inputs], 
  axis=0)
# print(np.shape(z_gradient_descent_history))
fig.add_trace(go.Scatter3d(x=theta_history[start:, 0], y=theta_history[start:, 1], 
                           z=z_gradient_descent_history + offset,
                           mode="lines",
                           line={"color": "red", "width": 3}, name="GD"))

z_minibatch_history = np.mean(
  [(theta_minibatch_history[start:, 0]*x + theta_minibatch_history[start:, 1] - y)**2 for x, y in inputs],
  axis=0)
fig.add_trace(go.Scatter3d(x=theta_minibatch_history[start:, 0], y=theta_minibatch_history[start:, 1],
                           z=z_minibatch_history + offset,
                           mode="lines",
                           line={"color": "blue", "width": 3}, name="Minibatch"))

z_sgd_history = np.mean(
  [(theta_sgd_history[start:, 0]*x + theta_sgd_history[start:, 1] - y)**2 for x, y in inputs],
  axis=0)
fig.add_trace(go.Scatter3d(x=theta_sgd_history[start:, 0], y=theta_sgd_history[start:, 1],
                           z=z_sgd_history + offset,
                           mode="lines",
                           line={"color": "green", "width": 3}, name="SGD"))

r = 1.6
camera = {"eye": {"x": r, "y": r, "z": r}}
fig.update_layout(autosize=False, width=500, height=400,
                  scene={"xaxis_title": "slope",
                         "yaxis_title": "intercept",
                         "zaxis_title": "f(slope, intercept)"},
                  margin={"l":5, "r":10, "b":5, "t":10},
                  scene_camera=camera,
                  legend={"x": 0.1, "y": 0.9})

fig.show()

